# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Setup

Import the json file into MongoDB using the following command

`mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# Confirm that the new database was created
print(mongo.list_database_names())

['admin', 'config', 'epa', 'fruits_db', 'gardenDB', 'local', 'met', 'uk_food']


In [4]:
# Assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# Review the collections in new database
print(db.list_collection_names())

['establishments']


In [6]:
# Review a document in the establishment collection
pprint(db.establishments.find_one())

{'AddressLine1': '1 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'FirstLight Bar & Café',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.64041272672,
 'FHRSID': 1290215,
 'LocalAuthorityBusinessID': 'PI/000083029',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DH',
 'RatingDate': '2020-11-09T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64fb6f83a1f31cb0ecb19a42'),
 'geocode': {'latitude': '51.150625', 'longitude': '1.378231'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1290215',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '

In [8]:
# Assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked it to be included it the analysis. Add the restaurant "Penang Flavours" to the database and update its business type ID.

In [9]:
# Insert a new document into the collection
penang_flavours ={
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}
establishments.insert_one(penang_flavours)

In [10]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields.
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
fields = {"_id": 0, "BusinessType": 1, "BusinessTypeID": 1}

# Capture the results to a variable
results = establishments.find_one(query, fields)

# Prerry Print the result
pprint(results)


{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


In [11]:
# Update the new restaurant with the BusinessTypeID found above
establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set":
        {"BusinessTypeID": 1}
     }
)
# Confirm that the restaurant was updated
establishments.find_one({"BusinessName": "Penang Flavours"})

{'_id': ObjectId('64fb6fb461cc93f6787ff797'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

The magazine is not interested in any establishments in Dover. Check how many documents contain the Dover Local Authority, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [12]:
# Find the number of documents containing Dover Local Authority
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"There are {dover_count} establishments in Dover")

There are 994 establishments in Dover


In [13]:
# Remove any establishments within the Dover Local Authority
result = establishments.delete_many({"LocalAuthorityName": "Dover"})

# Print the number of documents deleted
print(f"{result.deleted_count} documents deleted.")

994 documents deleted.


In [14]:
# Verify the count of documents containing Dover Local Authority is 0
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"There are {dover_count} establishments in Dover")

There are 0 establishments in Dover


In [15]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one())

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64fb6f83a1f31cb0ecb19d27'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          '

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [17]:
# Change the data type from String to Decimal for longitude and latitude
result = establishments.update_many({}, [ {'$set':{ "geocode.longitude" : {'$toDecimal': "$geocode.longitude"},
                                           "geocode.latitude" : {'$toDecimal': "$geocode.latitude"}
                                         }} ])

# Print the number of documents updated
print(f"{result.modified_count} documents updated.")

pprint(establishments.find_one())


0 documents updated.
{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64fb6f83a1f31cb0ecb19d27'),
 'geocode': {'latitude': Decimal128('51.086058'),
             'longitude': Decimal128('1.196408')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            

Use `update_many` to convert `RatingValue` to integer numbers.

In [20]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
results= establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

# Print the number of documents updated
print(f"{results.modified_count} documents updated.")

0 documents updated.


In [21]:
# Change the data type from String to Integer for RatingValue
result = establishments.update_many({}, [ {'$set':{ "RatingValue" : {'$toInt': "$RatingValue"}}} ])
# Print the number of documents updated
print(f"{result.modified_count} documents updated.")

34660 documents updated.


In [24]:
# Check that the coordinates and rating value are now numbers
query = {}
fields = {"geocode": 1, "RatingValue": 1}
establishments.find_one(query, fields)

{'_id': ObjectId('64fb6f83a1f31cb0ecb19d27'),
 'RatingValue': 4,
 'geocode': {'longitude': Decimal128('1.196408'),
  'latitude': Decimal128('51.086058')}}